Import packages and read in the data. Additionally, I chage the datatype of the totalcharges column.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_raw = pd.read_csv('../../../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [3]:
df_raw['TotalCharges'] = pd.to_numeric(df_raw['TotalCharges'], errors='coerce')

In [4]:
df_raw['SeniorCitizen'] = df_raw['SeniorCitizen'].map(lambda x: 'No' if x == 0 else 'Yes')

NameError: name 'df' is not defined

Most columns are categorical and are one of two or three options

In [ ]:
df_raw.head()

In [ ]:
#label encode Churn
#binary encode the yes/no
#one hot encode the rest

Make the column names easier to use and print out the number of unique values per column.

In [ ]:
df_raw.columns = df_raw.columns.str.lower()

Pull out the customer id's because I don't want to lose the data, but I also don't want to graph it. I also engineered two new features from the few numerical columns. The first is the total charges a customer incured divided by how long they have had the service. I thought that people who were charged a lot in a short time might churn more. I also divided monthly charges by total charges so that higher ratios relate to higher average monthly charges.

In [ ]:
cust_id = df_raw['customerid']
# df_raw['monchg_per_tenure'] = df_raw['monthlycharges'] / df_raw['tenure']
df_raw['totchg_per_tenure'] = df_raw['totalcharges'] / df_raw['tenure']
df_raw['monthly_div_tot'] = df_raw['monthlycharges'] / df_raw['totalcharges']

df_raw['partner_cont'] = df_raw['partner'].map({'No': 0, 'Yes': 1, })
df_raw['dependent_cont'] = df_raw['dependents'].map({'No': 0, 'Yes': 1, })
df_raw['single_parent'] = df_raw['partner_cont'] + df_raw['dependent_cont']
df_raw['single_parent'] = df_raw['single_parent'].map({1: 'No', 0: 'No', 2: 'Yes' })
df_raw.drop(['partner_cont', 'dependent_cont'], axis=1, inplace=True)
df_raw['phone_count'] = df_raw['phoneservice'].map({'No': 0, 'Yes': 1, })
df_raw['internet_count'] = df_raw['internetservice'].map({'No': 0, 'DSL': 1, 'Fiber optic': 1 })
df_raw['phone_and_internet'] = df_raw['phone_count'] + df_raw['internet_count']
df_raw['phone_and_internet'] = df_raw['phone_and_internet'].map({1: 'No', 0: 'No', 2: 'Yes' })
df_raw.drop(['phone_count', 'internet_count'], axis=1, inplace=True)


df = df_raw.drop('customerid',axis=1)


In [ ]:
for col in df_raw.columns:
    print(col,df_raw[col].nunique())

I need to encode this data to use it in models so I made lists for each type to do.

In [ ]:
ord_encode = ['gender','partner','dependents','phoneservice','multiplelines','internetservice','onlinesecurity','onlinebackup',
              'deviceprotection','techsupport','streamingtv','streamingmovies','contract','paperlessbilling','paymentmethod','single_parent','phone_and_internet','seniorcitizen']
labelencode =['churn']
num = ['tenure','monthlycharges','totalcharges','totchg_per_tenure','monthly_div_tot']

In [ ]:
X = df[ord_encode].join(df[num],how='left')
y = df[labelencode]



There are some missing values that need to be filled in total charges. I am using 0 because those customers probably have not been charged yet.

In [ ]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='constant',fill_value=0)

X_imp = impute.fit_transform(X)
X_proc = pd.DataFrame(X_imp,columns=X.columns)


In [ ]:
# from sklearn.preprocessing import OrdinalEncoder
# #ord encode categoticals
# ordi = OrdinalEncoder(dtype=np.int)

# X_proc = ordi.fit_transform(X_proc)
# X_enc = pd.DataFrame(X_proc,columns=X.columns)


Encode churn so that the counts of yes and no can be summed easier.

In [ ]:
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()

y_enc = lbl.fit_transform(np.array(y))

y_enc = pd.DataFrame(y_enc,columns=y.columns)

In [ ]:
y_enc

Join the feature and target frames and then I added a counter column which helps with some groupby stuff.

In [ ]:
df_enc = X.join(y_enc,how='left')

In [ ]:
df_raw['counter'] = 1
df_raw.groupby(by=['paymentmethod','churn']).sum()

looks like larger totchg_per_tenure is related to churn. Also half of check payment methods churn. Below, I make new dataframes for churn of not. Then, I can make plots right next to each other because I couldn't figure out how to put them in one chart.

In [ ]:
churned = df_raw.loc[df_raw['churn'] == 'Yes']
not_churned = df_raw.loc[df_raw['churn'] == 'No']


I started by defining the columns that I wanted to plot and taking out features that don't need to be plotted. I used a count plot for categoricals and a hist plot for numeric data because it runs faster and the bins automatically format.

In [ ]:
interesting_cols = ['partner','dependents','internetservice','onlinesecurity','onlinebackup',
              'deviceprotection','techsupport','contract','paperlessbilling','paymentmethod','single_parent','phone_and_internet','seniorCitizen']

numerics = ['tenure','monthlycharges','totalcharges','totchg_per_tenure','monthly_div_tot']

for col in churned[interesting_cols]:
    fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(15,6))

    sns.countplot(x=col,data=churned,ax=ax[0],color='cornflowerblue',orient='h')
    sns.countplot(x=col,data=not_churned,ax=ax[1],color='lightcoral',orient='h')
    ax[0].title.set_text(f'Churned - {col} ')
    ax[1].title.set_text(f'Not Churned - {col} ')
    
for col in churned[numerics]:
    fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(15,6))

    sns.histplot(x=col,data=churned,ax=ax[0],color='cornflowerblue',bins=10)
    sns.histplot(x=col,data=not_churned,ax=ax[1],color='lightcoral',bins=10)
    ax[0].title.set_text(f'Churned - {col} ')
    ax[1].title.set_text(f'Not Churned - {col} ')
    


From these graphs, more people without dependents dropped their service. Also, those who hadn't been with the company long (tenure) dropped more. Most drops were month to month contracts and electronic checks. Additionally, people without a large total charge dropped more. This makes sense because people who had a low tenure wouldn't be able to get a lot of total charges.

Those who got charged a lot in a short amount of time churned more.